In [1]:
import pandas as pd
import numpy as np

In [2]:
import plotly.express as px
import matplotlib as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
link = "/content/drive/MyDrive/train.csv"
df = pd.read_csv(link)

In [5]:
display(df.head())

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [6]:
df.shape

(27480, 4)

2. Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?

In [7]:
df_filtered = df[df['sentiment'] != 'neutral']

In [8]:
print(((df_filtered["sentiment"].value_counts() / len(df_filtered["sentiment"])) * 100).round(2))

sentiment
positive    52.45
negative    47.55
Name: count, dtype: float64


3. Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32

In [41]:
X = df_filtered["text"]
y = df_filtered["sentiment"]

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)

print(len(X_train))
print(len(X_test))

12272
4091


4. Crée un modèle vectorizer avec scikit-learn en utilisant la méthode TfidfVectorizer.

Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV.

Crée la matrice X_test_CV sans ré-entraîner le modèle.

Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [48]:
X_train_CV = tfidf.fit_transform(X_train)
X_train_CV

<12272x15806 sparse matrix of type '<class 'numpy.float64'>'
	with 144578 stored elements in Compressed Sparse Row format>

In [49]:
# ici j'utilise un CountVectorizer car il est demandé de ne pas entrainer le set de test
# or tfidf ne semble pas proposer cette possibilité

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

X_test_CV = vectorizer.transform(X_test)
X_test_CV

<4091x15806 sparse matrix of type '<class 'numpy.int64'>'
	with 44633 stored elements in Compressed Sparse Row format>

5. Entraîne maintenant une régression logistique avec les paramètres par défaut.

Tu devrais obtenir les résultats suivants : 0.932pour le test d'entraînement, et 0.873 pour l'ensemble de test.

In [50]:
X_train_M = X_train_CV.toarray()
X_test_M = X_test_CV.toarray()

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model_lg = LogisticRegression().fit(X_train_M, y_train)

In [52]:
# evaluation du modèle sur le set d'entrainement

y_pred_train = model_lg.predict(X_train_M)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Précision sur l'ensemble d'entraînement: {accuracy_train:.3f}")

Précision sur l'ensemble d'entraînement: 0.932


In [53]:
# evaluation du modèle sur le set de test

y_pred_test = model_lg.predict(X_test_M)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Précision sur l'ensemble de test: {accuracy_test:.3f}")

Précision sur l'ensemble de test: 0.857


In [ ]:
# je devrais obtenir 0.873 pour l'ensemble de test